In [1]:
import os
import h5py
import numpy as np
import pandas as pd
from lightfm import LightFM
from lightfm.data import Dataset
import pickle

c:\Users\ritac\anaconda3\envs\rec_training\Lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [ ]:
DATASET_PATH = "../data/MillionSongSubset"

In [3]:
def safe_decode(value):
    """Decode bytes to string; leave numbers unchanged."""
    if isinstance(value, bytes):
        return value.decode('utf-8')
    elif isinstance(value, np.ndarray) and value.dtype.kind in {'S', 'O'}:
        return value[0].decode('utf-8')
    return value

def extract_track_info(h5_file):
    """Extract track info and audio features from a .h5 file."""
    with h5py.File(h5_file, "r") as f:
        # Metadata
        meta = f['metadata/songs'][0]
        artist_name = safe_decode(meta['artist_name'])
        genre = safe_decode(meta['genre']) if 'genre' in meta.dtype.names else "unknown"
        
        # Analysis features
        analysis = f['analysis/songs'][0]
        track_id = safe_decode(analysis['track_id'])
        tempo = analysis['tempo']
        loudness = analysis['loudness']
        duration = analysis['duration']
        key = analysis['key']
        mode = analysis['mode']
        time_signature = analysis['time_signature']
        
    return {
        "track_id": track_id,
        "artist_name": artist_name,
        "tempo": tempo,
        "loudness": loudness,
        "duration": duration,
        "key": key,
        "mode": mode,
        "time_signature": time_signature
    }

In [ ]:
tracks = []
for root, dirs, files in os.walk(DATASET_PATH):
    for file in files:
        if file.endswith(".h5"):
            file_path = os.path.join(root, file)
            try:
                info = extract_track_info(file_path)
                tracks.append(info)
            except Exception as e:
                print(f"Failed to read {file_path}: {e}")

df_tracks = pd.DataFrame(tracks)
print(f"Loaded {len(df_tracks)} tracks")

Loaded 10000 tracks
Total rows: 10000
Unique track_ids: 10000
using 10000 samples for training


In [ ]:
# prepare item features
all_item_features = set()
item_features_list = []

for row in df_tracks.itertuples(index=False):
    features = [
        f"tempo:{row.tempo}",
        f"loudness:{row.loudness}",
        f"duration:{row.duration}",
        f"key:{row.key}",
        f"mode:{row.mode}",
        f"time_signature:{row.time_signature}"
    ]
    all_item_features.update(features)
    item_features_list.append((row.track_id, features))

print(f"Created {len(all_item_features)} unique features")

Created 23344 unique features
Sample features being created: ['loudness:-14.897', 'loudness:-19.569', 'tempo:99.995', 'duration:220.08118', 'tempo:144.232', 'tempo:83.646', 'tempo:120.005', 'loudness:-8.295', 'tempo:124.577', 'tempo:93.856']


In [ ]:
dataset = Dataset()
dataset.fit(
    users=df_tracks['artist_name'].unique(),
    items=df_tracks['track_id'].tolist(),
    item_features=list(all_item_features)  # Make sure these are the feature strings
)

users = df_tracks['artist_name'].unique().tolist()  # using artists as users
items = df_tracks['track_id'].tolist()
user_item_pairs = list(df_tracks[['artist_name', 'track_id']].itertuples(index=False, name=None))
interactions, weights = dataset.build_interactions(user_item_pairs)
print('created user-item matrix')

(4412, 10000)
(10000, 33344)
(4412, 4412)


In [8]:
item_features = dataset.build_item_features(item_features_list)

In [ ]:
# model = LightFM(loss='bpr', no_components=15)
# model.fit(interactions, item_features=item_features, epochs=10, num_threads=1)